# Genetic algo for lighthouse sensor distribution on arbitrary mesh

In [1]:
import numpy as np
from stl import mesh as meshstl
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from data.plotmesh import plot_mesh
import math
import random
from pyquaternion import Quaternion
from scipy.linalg import qr
import roslib
import rospy
import math
import tf
rospy.init_node('fixed_tf_broadcaster')

In [2]:
#how many sensors would u like to distribute?
sensorsToDistribute = 10
stl_file = 'roboy_models/TestCube/stls/IcoSphere_360.stl'

In [3]:
#Move Lighthouses to
translationLH1 = [-2.,0,2.]
quat1 = Quaternion(axis=[0,0,1],angle=-np.pi / 2)

global LH1 
LH1 = (translationLH1, quat1)

translationLH2 = [2,0.,2.]
quat2 = Quaternion(axis=[0,0,1], angle=np.pi / 2)

global LH2
LH2 = (translationLH2, quat2)

print(LH1); print(LH2)

([-2.0, 0, 2.0], Quaternion(0.70710678118654757, -0.0, -0.0, -0.70710678118654746))
([2, 0.0, 2.0], Quaternion(0.70710678118654757, 0.0, 0.0, 0.70710678118654746))


In [4]:
from data.rvizMeshVis import meshVisualization

scale = 0.01
position = [0,0,0]
global orientationMesh
orientationMesh = Quaternion(axis=(1,0,0),angle = np.pi*0)

meshVisualization(orientationMesh, stl_file)

# Preprocess data

In [5]:
from data.trisByDistance import *

#Get mesh vertices and normals
mesh1 = meshstl.Mesh.from_file(('../'+ stl_file))
#mesh1 = meshstl.Mesh.from_file('../src/roboy_models/roboy_2_0_simplified/meshes/CAD/torso.stl')

global triangles
triangles = scale * np.matrix(mesh1.points)

global trianglesBackup
trianglesBackup = triangles

global sortedTriangles

lighthouses = [LH1, LH2]
sortedTriangles = []



for l in lighthouses:
    tris = trisByMinDistanceSortedMap(triangles, l[0])
    sortedTriangles.append(tris)


#for (j, dist) in sortedTriangles[1]:
#    print("Triangle %i %f" % (j, dist))

vertices = np.reshape(triangles,(len(triangles)*3,3)) 

#Initialize sensors in centers of triangle
sensors = (triangles[:,0:3]+triangles[:,3:6]+triangles[:,6:9])/3

print('%d triangles' %len(triangles))
print('')
print('%d vertices' %len(vertices))
print('')
print('%d sensors in centers of triangles' %len(sensors))

484 triangles

1452 vertices

484 sensors in centers of triangles


# GA

In [6]:
from deap import algorithms, base, creator, tools

In [7]:
#sensors to dict
global sensor_dict
sensor_dict =  list(zip(range(len(sensors)), sensors.tolist()))

global sensorDictBackup
sensorDictBackup = sensor_dict


In [8]:
from data.rvizSensorVis import sensorVisualization

#color = (r,g,b,a)
sensorVisualization(sensor_dict, rate=500, sphereSize=0.03, color=(0,0,1,1))

In [9]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,1.0)) # 1 -> maximum probblem
creator.create("Individual", list, fitness=creator.FitnessMax)

In [10]:
toolbox = base.Toolbox()

In [11]:
from data.randomSensor import randomSensor

toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", randomSensor, sensor_dict)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, sensorsToDistribute)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [12]:
toolbox.attr_bool()

451

In [13]:
toolbox.individual()

[244, 383, 82, 34, 199, 71, 232, 143, 6, 28]

# Evaluation (Fitness) Function

In [14]:
from data.RayIntersectsTriangle import rayIntersectsTriangle, rayIntersectsTriangleVisual

def FitnessFunction(sensors):
    
    br = tf.TransformBroadcaster()
    br.sendTransform((LH1[0][0], LH1[0][1], LH1[0][2]),
                     (quat1[0], quat1[1], quat1[2], quat1[3]),
                     rospy.Time.now(),
                     "lighthouse1",
                     "world")
    br.sendTransform((LH2[0][0], LH2[0][1], LH2[0][2]),
                     (quat2[0], quat2[1], quat2[2], quat2[3]),
                     rospy.Time.now(),
                     "lighthouse2",
                     "world")

    
    #1. COMPUTE VISIBLE SENSORS AT THE MOMENT
    LH1_array = np.asarray(LH1[0])
    LH2_array = np.asarray(LH2[0])
    #testTriangle = np.squeeze(np.asarray(triangles[0]))

    visibleLH1 = 0
    visibleLH2 = 0
    

    for i, nmbr_sensor in enumerate(sensors):
        sensor = sensor_dict[nmbr_sensor][1]

        #get distance of current sensor and check if intersection
        interDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                             np.squeeze(np.asarray(triangles[nmbr_sensor])), 'lighthouse1')
        distLH1 = np.linalg.norm(np.asarray(sensor) - LH1_array)
        interDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                             np.squeeze(np.asarray(triangles[nmbr_sensor])), 'lighthouse2')
        distLH2 = np.linalg.norm(np.asarray(sensor) - LH2_array)
        
        #print('interDist');print(interDistLH1);print(interDistLH2);print('endinterDist')

        isVisible1 = True
        isVisible2 = True
        
        
        # 1st lighthouse
        for (j, dist) in sortedTriangles[0]:
            if(nmbr_sensor != j):
                #print("Testing sensor %i vs tris %i: distance of Sensor %f vs triangle %f" % (i, j, distLH1, dist))
                tris = triangles[j]
                newInterDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                                    np.squeeze(np.asarray(tris)), 'lighthouse1')#,j)
                if(newInterDistLH1 < interDistLH1 and newInterDistLH1 != False):
                    isVisible1 = False
                if(not isVisible1 or dist > distLH1):
                    # Break if not visible or already checked all tris
                    # that are located closer to the lighthouse that the sensor
                    break
                    
        # 2nd lighthouse
        for (j, dist) in sortedTriangles[1]:
            if(nmbr_sensor != j):
                tris = triangles[j]
                newInterDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                                    np.squeeze(np.asarray(tris)), 'lighthouse2')#,j)
                if(newInterDistLH2 < interDistLH2 and newInterDistLH2 != False):
                    isVisible2 = False
                if(not isVisible2 or dist > distLH2):
                    # Break if not visible or already checked all tris
                    # that are located closer to the lighthouse that the sensor
                    break
        
        if(isVisible1):
            visibleLH1 += 1
        if(isVisible2):
            visibleLH2 += 1
        #print(newInterDistLH1); print(newInterDistLH2)
    
    fractionVisibleLH1 = float(visibleLH1) / sensorsToDistribute
    fractionVisibleLH2 = float(visibleLH2) / sensorsToDistribute
        
    #2. COMPUTE EUCLIDEAN DISTANCE OF SENSORS
    individual = sensors
    dist = 0
    for i,ind in enumerate(individual):
        ind = np.asarray(sensor_dict[ind][1])
        for j in range(i,len(individual)):
            if(i != j):
                indivi = np.asarray(sensor_dict[individual[j]][1])
                dist += np.linalg.norm(ind-indivi)

    #print(visibleLH1);print(visibleLH2);print('')
    
    return fractionVisibleLH1, fractionVisibleLH2

In [15]:
toolbox.register("evaluate", FitnessFunction)

toolbox.register("mate", tools.cxTwoPoint)
# Independent probability  : for each attribute to be mutated.# low~up rondom int
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(sensors.tolist())-1, indpb=0.2) 
toolbox.register("select", tools.selTournament, tournsize=3)

In [16]:
# Creating population

population = toolbox.population(n=10)

In [17]:
hof = tools.HallOfFame(1)

In [18]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [19]:
from data.algorithmsMod import varAnd
from deap import tools
from data.trafomatrix import getRandomRotationmatrix
from data.bestSensorVis import bestSensorVis

#MODDED VERSION of eaSimple from DEAP
def eaSimpleMod(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """This algorithm reproduce the simplest evolutionary algorithm as
    presented in chapter 7 of [Back2000]_.

    :param population: A list of individuals.
    :param toolbox: A :class:`~deap.base.Toolbox` that contains the evolution
                    operators.
    :param cxpb: The probability of mating two individuals.
    :param mutpb: The probability of mutating an individual.
    :param ngen: The number of generation.
    :param stats: A :class:`~deap.tools.Statistics` object that is updated
                  inplace, optional.
    :param halloffame: A :class:`~deap.tools.HallOfFame` object that will
                       contain the best individuals, optional.
    :param verbose: Whether or not to log the statistics.
    :returns: The final population
    :returns: A class:`~deap.tools.Logbook` with the statistics of the
              evolution

    The algorithm takes in a population and evolves it in place using the
    :meth:`varAnd` method. It returns the optimized population and a
    :class:`~deap.tools.Logbook` with the statistics of the evolution. The
    logbook will contain the generation number, the number of evalutions for
    each generation and the statistics if a :class:`~deap.tools.Statistics` is
    given as argument. The *cxpb* and *mutpb* arguments are passed to the
    :func:`varAnd` function. The pseudocode goes as follow ::

        evaluate(population)
        for g in range(ngen):
            population = select(population, len(population))
            offspring = varAnd(population, toolbox, cxpb, mutpb)
            evaluate(offspring)
            population = offspring

    As stated in the pseudocode above, the algorithm goes as follow. First, it
    evaluates the individuals with an invalid fitness. Second, it enters the
    generational loop where the selection procedure is applied to entirely
    replace the parental population. The 1:1 replacement ratio of this
    algorithm **requires** the selection procedure to be stochastic and to
    select multiple times the same individual, for example,
    :func:`~deap.tools.selTournament` and :func:`~deap.tools.selRoulette`.
    Third, it applies the :func:`varAnd` function to produce the next
    generation population. Fourth, it evaluates the new individuals and
    compute the statistics on this population. Finally, when *ngen*
    generations are done, the algorithm returns a tuple with the final
    population and a :class:`~deap.tools.Logbook` of the evolution.

    .. note::

        Using a non-stochastic selection method will result in no selection as
        the operator selects *n* individuals from a pool of *n*.

    This function expects the :meth:`toolbox.mate`, :meth:`toolbox.mutate`,
    :meth:`toolbox.select` and :meth:`toolbox.evaluate` aliases to be
    registered in the toolbox.

    .. [Back2000] Back, Fogel and Michalewicz, "Evolutionary Computation 1 :
       Basic Algorithms and Operators", 2000.
    """
    global sensor_dict
    global triangles
    global orientationMesh
    
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print logbook.stream

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Vary the pool of individuals
        offspring = varAnd(offspring, toolbox, cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print logbook.stream
        
        sensorMovement = tools.selBest(population, k=1)[0]
        bestSensorVis(sensor_dict, sensorMovement, rate=500, color=(0,1,0,0.8), sphereSize=0.3)
        
        if(gen%25==0):
            global sensorDictBackup
            global trianglesBackup
            sensor_dict = sensorDictBackup
            R = getRandomRotationmatrix()
            sensorDictNew = []
            
            for sensor in sensor_dict:
                sensorDictNew.append(np.squeeze(np.asarray(R.dot(np.array(sensor[1])))).tolist())
                
            sensor_dict = list(zip(range(len(sensorDictNew)), sensorDictNew))
        
            tri1 = R.dot(np.transpose(trianglesBackup[:,0:3]))
            tri2 = R.dot(np.transpose(trianglesBackup[:,3:6]))
            tri3 = R.dot(np.transpose(trianglesBackup[:,6:9]))

            triangles = np.concatenate((tri1.T,tri2.T,tri3.T),axis=1)
            
            # resort the triangles by distance from lighthouses for speedup
            global sortedTriangles

            lighthouses = [LH1, LH2]
            sortedTriangles = []

            for l in lighthouses:
                tris = trisByMinDistanceSortedMap(triangles, l[0])
                sortedTriangles.append(tris)
                
                

            orientationMesh = Quaternion(matrix=R)
            
            meshVisualization(orientationMesh, stl_file)
            sensorVisualization(sensor_dict, rate=500, sphereSize=0.03, color=(0,0,1,1))
            
            
    return population, logbook

In [20]:
population, log = eaSimpleMod(population, 
                                toolbox, 
                                cxpb=0.5, 
                                mutpb=0.2, 
                                ngen=100, 
                                stats=stats, 
                                halloffame=hof, 
                                verbose=True)

gen	nevals	avg 	std     	min	max
0  	10    	0.34	0.162481	0  	0.7
1  	4     	0.425	0.166958	0.1	0.6
2  	6     	0.475	0.144482	0.2	0.6
3  	8     	0.515	0.115217	0.3	0.7
4  	8     	0.575	0.117792	0.3	0.8
5  	6     	0.605	0.143091	0.2	0.8
6  	3     	0.635	0.149248	0.4	0.8
7  	3     	0.625	0.166958	0.3	0.8
8  	6     	0.64 	0.149666	0.4	0.8
9  	4     	0.63 	0.155242	0.4	0.8
10 	6     	0.65 	0.15    	0.5	0.8
11 	8     	0.635	0.14586 	0.4	0.8
12 	8     	0.645	0.143091	0.4	0.8
13 	7     	0.655	0.111692	0.5	0.8
14 	10    	0.67 	0.122882	0.4	0.8
15 	4     	0.66 	0.131909	0.4	0.8
16 	7     	0.67 	0.114455	0.5	0.8
17 	6     	0.68 	0.107703	0.5	0.8
18 	0     	0.7  	0.1     	0.6	0.8
19 	6     	0.7  	0.0948683	0.6	0.8
20 	8     	0.71 	0.0943398	0.6	0.8
21 	6     	0.71 	0.0830662	0.6	0.8
22 	5     	0.69 	0.126095 	0.3	0.8
23 	1     	0.735	0.0653835	0.6	0.8
24 	6     	0.73 	0.0714143	0.6	0.8
25 	8     	0.71 	0.0994987	0.5	0.8
26 	4     	0.585	0.257439 	0  	0.8
27 	10    	0.35 	0.241868 	0.1	0.6
28 	4  

In [21]:
bestSensors = tools.selBest(population, k=1)[0]
print(bestSensors)

[172, 443, 152, 454, 154, 209, 441, 150, 156, 233]


In [22]:
from data.bestSensorVis import bestSensorVis
#Sensor visualization in RVIZ

bestSensorVis(sensor_dict, bestSensors, rate=500, color=(1,0,0,0.8), sphereSize=0.1)
